In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, countDistinct, to_date
from pyspark.sql.functions import lit
from pyspark.sql.functions import  from_json, from_unixtime, date_format

In [ ]:
%md #Initialize Spark session

In [ ]:
spark = SparkSession.builder \
    .appName("User Click Data ETL") \
    .getOrCreate()
